In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import re
import math
import seaborn as sns
import scipy.stats as stats
from datetime import datetime, timedelta, date
from fitter import Fitter, get_common_distributions, get_distributions

In [ ]:
# options
pd.set_option('max_columns',100)
warnings.simplefilter('ignore')
plt.style.use('seaborn-whitegrid')

In [ ]:
save_dir = '../result/notebook/'

# 내꺼

In [ ]:
# initialize
absolute_difference = lambda datalist : abs(datalist - pressure_altitude)

to = pd.DataFrame()
ld = pd.DataFrame()
others = pd.DataFrame()

In [ ]:
# load data
daterange = pd.date_range(start='12/21/2019 15:00', end='12/28/2019 14:00', freq='1H').strftime('%Y%m%d_%H%M%S')  # Zulu time
dff = pd.DataFrame()

for i in range(12,14):
    date = daterange[i]
    print(date)
    df = pd.read_csv(f'../input/MLAT/tp1_{date}_Cat20.ast_Mlat.csv', index_col=False, header = 1)
    dff = pd.concat([dff, df])

# 여기서 date를 concat하면 같은 c/s인 항공기가 있어서 안 좋을 것 같음 -> 가장 큰 for loop으로 쓰기
# for date in daterange:
#    df = pd.read_csv(f'../input/tp1_{i}_Cat20.ast_Mlat.csv', index_col=False, header = 1)

In [ ]:
# calculate pressure altitude

current_pressure = float(input('Current pressure : '))
fe_m = 7
msl_ft = 145366.45 * (1 - (current_pressure/1013.25)**0.190284)
msl_m = msl_ft * 0.3048
pressure_altitude = msl_m + fe_m
print('RKSI Field Elevation : 7 m')
print(f'Pressure Altitude : {pressure_altitude} m')

In [ ]:
# Wheel On/Off 지점 찾기

ident = np.unique(df['ModeSIdent'].values).tolist()


for i in ident:
    # c/s별로 wheel on/off 지점인 데이터만 넣기
    temp_df = df[df['ModeSIdent'] == i]
    temp_df['BAlt'] = temp_df['BAlt'].replace('----', 10000).astype(float)         # '----'값은 의미없도록 10000으로 바꿈
    temp_altitude_list = temp_df['BAlt']
    temp_closest_value = min(temp_altitude_list, key = absolute_difference)        # absolute_difference 으로 가장 가까운 값 찾기
    temp_df = temp_df[temp_df['BAlt'] == temp_closest_value]
    
    # ops로 arr / dep나누기
    if temp_df['GroudSpeed'].iloc[0] < temp_df['GroudSpeed'].iloc[-1]:
        temp_df['ops'] = 'departure'
    elif temp_df['GroudSpeed'].iloc[0] > temp_df['GroudSpeed'].iloc[-1]:
        temp_df['ops'] = 'arrival'
    else:
        temp_df['ops'] = 'others'
    
    # arr / dep 각각 저장 - # landing은 0, takeoff는 -1로 해야함
    if abs(temp_df['BAlt'].iloc[0] - pressure_altitude) > 100:    # 너무 큰 이상한 것은 버림
        others = others.append(temp_df)
    elif (temp_df['ops'].iloc[0] == 'departure') & (temp_df['GroudSpeed'].iloc[-1] > 50):    
        to = to.append(temp_df.iloc[-1,:])    
    elif (temp_df['ops'].iloc[0] == 'arrival') & (temp_df['GroudSpeed'].iloc[0] > 50):
        ld = ld.append(temp_df.iloc[0,:])
    else:
        others = others.append(temp_df)

In [ ]:
base_map = plt.imread('../data/map.png')
BBox = (126.3940, 126.4743, 37.4414, 37.4856)

fig, ax = plt.subplots(figsize = (25,20))
ax.scatter(to['Longitude'], to['Latitude'], alpha= 0.6, c='b', s=15, marker = 'o')
ax.scatter(ld['Longitude'], ld['Latitude'], alpha= 0.6, c='r', s=15, marker = 'x')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(base_map, zorder = 0, extent = BBox, aspect= 'equal')

### Go Around 찾기

In [ ]:
dff[dff['ModeSIdent']  == 'AAR131']



# 교수님 데이터

In [ ]:
def datenum_to_datetime(datenum):
    """
    Convert Matlab datenum into Python datetime.
    :param datenum: Date in datenum format
    :return:        Datetime object corresponding to datenum.
    """
    days = datenum % 1
    hours = days % 1 * 24
    minutes = hours % 1 * 60
    seconds = minutes % 1 * 60
    return datetime.fromordinal(int(datenum)) \
           + timedelta(days=int(days)) \
           + timedelta(hours=int(hours)) \
           + timedelta(minutes=int(minutes)) \
           + timedelta(seconds=round(seconds)) \
           - timedelta(days=366)

In [ ]:
# RWY : '33L', '33R', 34, '15R', '15L', 16  -->>  1-3 : Northflow / 4-6 : Southflow

columns = ['Time', 'Callsign', 'AC', 'Lat', 'Lon', 'RWY', 'DIST', 'ROT1', 'ROT2']

In [ ]:
arr_df = pd.read_csv(f'../input/arr_data.csv', index_col=False, header = 0)
dep_df = pd.read_csv(f'../input/dep_data.csv', index_col=False, header = 0)
freetime_df = pd.read_csv(f'../input/freetime.csv', index_col=False, header = 0)

In [ ]:
arr_df.columns = columns
dep_df.columns = columns
arr_df['Time']= pd.to_datetime(arr_df['Time'])
dep_df['Time']= pd.to_datetime(dep_df['Time'])

In [ ]:
arr_df_north = arr_df[arr_df['RWY'] <= 3]
dep_df_north = dep_df[dep_df['RWY'] <= 3]
arr_df_south = arr_df[arr_df['RWY'] > 3]
dep_df_south = dep_df[dep_df['RWY'] > 3]

In [ ]:
freetime = pd.DataFrame()
for j in range(0,35,7):
    temp = pd.concat([freetime_df[i] for i in freetime_df.columns[j:j+7]]).reset_index(drop = True)
    freetime = pd.concat([freetime, pd.DataFrame(temp)], axis = 1)
freetime.columns = ['Time', 'Total', 'Num', 'Mean', 'Std']

## Scatter on the map

In [ ]:
base_map = plt.imread('../data/map.png')
BBox = (126.3940, 126.4743, 37.4414, 37.4856)

In [ ]:
# North flow

fig, ax = plt.subplots(figsize = (25,20))
ax.scatter(dep_df_north['Lon'], dep_df_north['Lat'], alpha= 0.6, c='b', s=30, marker = 'o', label = 'Departure')
ax.scatter(arr_df_north['Lon'], arr_df_north['Lat'], alpha= 0.6, c='r', s=30, marker = 'x', label = 'Arrival')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.legend(fontsize=20)
ax.imshow(base_map, zorder = 0, extent = BBox, aspect= 'equal')

plt.savefig(save_dir + 'northflow.png', bbox_inches='tight', pad_inches=1)

In [ ]:
# South flow
fig, ax = plt.subplots(figsize = (25,20))
ax.scatter(dep_df_south['Lon'], dep_df_south['Lat'], alpha= 0.6, c='b', s=30, marker = 'o', label = 'Departure')
ax.scatter(arr_df_south['Lon'], arr_df_south['Lat'], alpha= 0.6, c='r', s=30, marker = 'x', label = 'Arrival')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.legend(fontsize=20)
ax.imshow(base_map, zorder = 0, extent = BBox, aspect= 'equal')

plt.savefig(save_dir + 'southflow.png', bbox_inches='tight', pad_inches=1)

In [ ]:
plt.figure(figsize=(15, 15))

# northflow
plt.subplot(2,1,1)
sns.histplot(data = arr_df_north, x="DIST", kde=True, hue="RWY", multiple='layer', bins = 50)
plt.annotate(round(np.mean(arr_df_north['DIST'])), 
             xy = (np.mean(arr_df_north['DIST']), 10),
             xytext = (np.mean(arr_df_north['DIST']), 0))

plt.subplot(2,1,1)
sns.histplot(data = dep_df_north, x="DIST", kde=True, hue="RWY", multiple='layer', bins = 50)
plt.annotate(round(np.mean(dep_df_north['DIST'])), 
             xy = (np.mean(dep_df_north['DIST']), 10),
             xytext = (np.mean(dep_df_north['DIST']), 0))
plt.legend(labels = [ '33L', '33R', '34', '33L', '33R', '34'])
plt.axvline(np.mean(arr_df_north['DIST']), color='crimson', linestyle = '--', linewidth = 2, label = 'mean')
plt.axvline(np.mean(dep_df_north['DIST']), color='crimson', linestyle = '--', linewidth = 2, label = 'mean')
plt.title('North')


# southflow
plt.subplot(2,1,2)
sns.histplot(data = arr_df_south, x="DIST", kde=True, hue="RWY", multiple='layer', bins = 50)
plt.annotate(round(np.mean(arr_df_south['DIST'])), 
             xy = (np.mean(arr_df_south['DIST']), 10),
             xytext = (np.mean(arr_df_south['DIST']), 0))

plt.subplot(2,1,2)
sns.histplot(data = dep_df_south, x="DIST", kde=True, hue="RWY", multiple='layer', bins = 50)
plt.annotate(round(np.mean(dep_df_south['DIST'])), 
             xy = (np.mean(dep_df_south['DIST']), 10),
             xytext = (np.mean(dep_df_south['DIST']), 0))
plt.legend(labels = ['15R', '15L', '16', '15R', '15L', '16'])
plt.axvline(np.mean(arr_df_south['DIST']), color='crimson', linestyle = '--', linewidth = 2, label = 'mean')
plt.axvline(np.mean(dep_df_south['DIST']), color='crimson', linestyle = '--', linewidth = 2, label = 'mean')
plt.title('South')

plt.savefig(save_dir + 'entire_dist.png', bbox_inches='tight', pad_inches=1)

***
***
***

## Aircraft Weight Class

payload를 알 수 있다면 gross weight으로 적용하고, 더 정확할 듯

<b>FAA Order JO 7360.1E Aircraft Type Designators</b> <br>
RECAT 2.0 Wake Category apdx.B <br>

(1) Category A – Upper Heavy Aircraft <br>
(2) Category B – Lower Heavy Aircraft <br>
(3) Category C – Upper Large Aircraft <br>
(4) Category D – Lower Large Aircraft <br>
(5) Category E – Small Plus aircraft with a maximum takeoff weight of more than 15,400 pounds up to 41,000 pounds <br>
(6) Category F – Small aircraft with a maximum takeoff weight of 15,400 pounds or less <br>
(7) Category G – Heavy aircraft not included in Category A or B <br>

In [ ]:
# dep_df_south['AC'].value_counts()

mapping = {'B738':'C', 'A321':'C', 'A333':'A', 'A320':'C', 'B772':'A', 'B77W':'A', 'B744':'A',
           'B748':'A', 'A359':'A', 'B739':'C', 'A332':'A', 'B789':'A', 'A21N':'C', 'B763':'B',
           'B773':'A', 'B77L':'A', 'B737':'C', 'B788':'A', 'A20N':'C', 'A319':'C', 'B787':'A',
           'A339':'G', 'B767':'B', 'MD11':'B', 'B78X':'G', 'A306':'C', 'B747':'A', 'A330':'A', 
           'B752':'C', 'B777':'A', 'A124':'G', 'A388':'NOWGT', 'A380':'NOWGT'}

In [ ]:
AC_cat = ['NOWGT', 'A', 'B', 'G', 'C']

In [ ]:
# 기종에서 무게 구분으로 바꾸기
dep_df_north.loc[:,'AC'] = dep_df_north['AC'].map(mapping)
dep_df_south.loc[:,'AC'] = dep_df_south['AC'].map(mapping)
arr_df_north.loc[:,'AC'] = arr_df_north['AC'].map(mapping)
arr_df_south.loc[:,'AC'] = arr_df_south['AC'].map(mapping)
arr_df.loc[:,'AC'] = arr_df['AC'].map(mapping)
dep_df.loc[:,'AC'] = dep_df['AC'].map(mapping)

### Distribution fitting

In [ ]:
sns.displot(data = arr_df, x = "DIST", kde = True, color = 'r', kind="hist", bins = 100, aspect = 1.5)
sns.displot(data = dep_df, x = "DIST", kde = True, color = 'b', kind="hist", bins = 100, aspect = 1.5)

In [ ]:
# Arrival

dist = arr_df['DIST'].values
f_arrival_total = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
f_arrival_total.fit()
print('Arrival Total Distribution')
print(f_arrival_total.summary())
plt.show()

weight_class = arr_df['AC'].unique()[:-1]

arrival_distribution = {}

for clss in weight_class:    
    temp_df = arr_df[arr_df['AC'] == f'{clss}']
    dist = temp_df['DIST'].values
    arrival_distribution[clss] = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
    arrival_distribution[clss].fit()
    print(clss)
    print(arrival_distribution[clss].summary())
    plt.show()

In [ ]:
# Departure

dist = dep_df['DIST'].values
f_departure_total = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
f_departure_total.fit()
print('Departure Total Distribution')
print(f_departure_total.summary())
plt.show()

weight_class = np.delete(dep_df['AC'].unique(), 3)    # remove nan

departure_distribution = {}

for clss in weight_class:    
    temp_df = dep_df[dep_df['AC'] == f'{clss}']
    dist = temp_df['DIST'].values
    departure_distribution[clss] = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
    departure_distribution[clss].fit()
    print(clss)
    print(departure_distribution[clss].summary())
    plt.show()

"""
# Departure

dist = dep_df['DIST'].values
f = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
f.fit()
print('Departure Total Distribution')
print(f.summary())

weight_class = np.delete(dep_df['AC'].unique(), 3)    # remove nan

for clss in weight_class:
    temp_df = dep_df[dep_df['AC'] == f'{clss}']
    dist = temp_df['DIST'].values
    f = Fitter(dist, distributions = get_common_distributions(), timeout = 100)
    f.fit()
    print(clss)
    print(f.summary())"""

In [ ]:
# f_departure_total  /  departure_distribution['key']  /  f_arrival_total  /  arrival_distribution['key']
# 'A', 'C', 'NOWGT', 'B', 'G'

# show result
print(f_departure_total.summary())
#print(arrival_distribution['A'].summary())
plt.show()

# get parameters
f_departure_total.fitted_param['norm']
#arrival_distribution['A'].fitted_param["rayleigh"]    # arrival_distribution['A'].get_best(method = 'sumsquare_error')

In [ ]:
print(f_departure_total.fitted_param["norm"])


print(departure_distribution['NOWGT'].fitted_param["norm"])
print(departure_distribution['A'].fitted_param["norm"])
print(departure_distribution['B'].fitted_param["norm"])
print(departure_distribution['G'].fitted_param["norm"])
print(departure_distribution['C'].fitted_param["norm"])


In [ ]:
# ANOVA
F_statistic, pVal = stats.f_oneway(arr_df[arr_df['AC'] == 'A']['DIST'],
                                   arr_df[arr_df['AC'] == 'B']['DIST'],
                                   arr_df[arr_df['AC'] == 'C']['DIST'],
                                   arr_df[arr_df['AC'] == 'G']['DIST'],
                                   arr_df[arr_df['AC'] == 'NOWGT']['DIST'],
                                  )

print('result : F={0:.1f}, p={1:.5f}'.format(F_statistic, pVal))

In [ ]:
mu = 1808.65969437491
variance = 366.58176065603374
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.show()

### Wheel On/Off

In [ ]:
# south

fig, axs = plt.subplots(5,5, figsize=(50, 50), squeeze = False)

for i, dep_category in enumerate(AC_cat):
    temp_dep = dep_df_south[dep_df_south['AC'] == f'{dep_category}']
    for j, arr_category in enumerate(AC_cat):
        temp_arr = arr_df_south[arr_df_south['AC'] == f'{arr_category}']
        sns.histplot(ax = axs[j, 4-i], data = temp_dep, x = "DIST", kde = True, color = 'navy')
        sns.histplot(ax = axs[j, 4-i], data = temp_arr, x = "DIST", kde = True, color = 'r')
        axs[j, 4-i].axvline(np.mean(temp_dep['DIST']), color='crimson', linestyle = '--', linewidth = 2)
        axs[j, 4-i].axvline(np.mean(temp_arr['DIST']), color='crimson', linestyle = '--', linewidth = 2)
        axs[j, 4-i].set_title(f'Dep : {dep_category} - Arr : {arr_category}')
fig.suptitle('Southflow Wheel On/Off with A/C Weight Class', fontsize = 80)
plt.savefig(save_dir + 'south_Wheel.png', bbox_inches='tight', pad_inches=1)
# hue = 'RWY' 넣으면 rwy에 따라서도 볼 수 있음 -> 보기 좋지는 않음

In [ ]:
# north

fig, axs = plt.subplots(5,5, figsize=(50, 50))

for i, dep_category in enumerate(AC_cat):
    temp_dep = dep_df_north[dep_df_north['AC'] == f'{dep_category}']
    for j, arr_category in enumerate(AC_cat):
        temp_arr = arr_df_north[arr_df_north['AC'] == f'{arr_category}']
        sns.histplot(ax = axs[j, 4-i], data = temp_dep, x = "DIST", kde = True, color = 'navy')
        sns.histplot(ax = axs[j, 4-i], data = temp_arr, x = "DIST", kde = True, color = 'r')
        axs[j, 4-i].axvline(np.mean(temp_dep['DIST']), color='crimson', linestyle = '--', linewidth = 2)
        axs[j, 4-i].axvline(np.mean(temp_arr['DIST']), color='crimson', linestyle = '--', linewidth = 2)
        axs[j, 4-i].set_title(f'Dep : {dep_category} - Arr : {arr_category}')
fig.suptitle('Northflow Wheel On/Off with A/C Weight Class', fontsize = 80)
plt.savefig(save_dir + 'north_Wheel.png', bbox_inches='tight', pad_inches=1)

# hue = 'RWY' 넣으면 rwy에 따라서도 볼 수 있음 -> 보기 좋지는 않음

### Runway Occupancy Time

In [ ]:
def _freetime_(arr_df, dep_df):

    # 33L/15R, 33R/15L에 대한 항공기
    rightrwy_arr = arr_df[(arr_df['RWY'] == 1) | (arr_df['RWY'] == 2) | (arr_df['RWY'] == 4) | (arr_df['RWY'] == 5)].loc[:,'ROT1':'ROT2']
    rightrwy_dep = dep_df[(dep_df['RWY'] == 1) | (dep_df['RWY'] == 2) | (dep_df['RWY'] == 4) | (dep_df['RWY'] == 5)].loc[:,'ROT1':'ROT2']


    # total Runway Occupancy Time
    totalROT = rightrwy_arr.append(rightrwy_dep).reset_index(drop = True) # totalROT
    totalROT['ROT2'] = totalROT['ROT2'].sort_values(ignore_index = True)
    totalROT['ROT1'] = totalROT['ROT1'].sort_values(ignore_index = True)


    # Occupied Time
    occupiedTime = pd.DataFrame(totalROT.iloc[0,:]).T
    occupiedTime.columns = ['ROT1', 'ROT2']
    nInterval = 0

    for i in range(1,len(totalROT)):
        if occupiedTime['ROT2'][nInterval] >= totalROT['ROT1'][i]:
            if occupiedTime['ROT2'][nInterval] < totalROT['ROT2'][i]:
                occupiedTime['ROT2'][nInterval] = totalROT['ROT2'][i]
        else:
            occupiedTime = occupiedTime.append(pd.DataFrame(np.zeros((1,2)), columns = ['ROT1', 'ROT2']))
            nInterval = nInterval + 1
            occupiedTime.iloc[nInterval, :] = totalROT.iloc[i,:]
            occupiedTime = occupiedTime.reset_index(drop = True)

    occupiedTime['ROT1'] = occupiedTime['ROT1'].apply(datenum_to_datetime)
    occupiedTime['ROT2'] = occupiedTime['ROT2'].apply(datenum_to_datetime)


    # freetime dataframe init
    freeTimeDuration = pd.DataFrame(np.zeros((nInterval, 1)))
    freeTimeDay = pd.DataFrame(np.zeros((nInterval, 1)))
    freeTimeHour = pd.DataFrame(np.zeros((nInterval, 1)))

    for i in range(1,nInterval+1):
        freeTimeDuration.iloc[i-1,0] = (occupiedTime['ROT1'][i] - occupiedTime['ROT2'][i-1]).seconds
        freeTimeDay.iloc[i-1,0] = occupiedTime['ROT2'][i-1].day
        freeTimeHour.iloc[i-1,0] = occupiedTime['ROT2'][i-1].hour

    temp_freeTimeTot = pd.DataFrame(np.zeros((24, int(freeTimeDay.max()-freeTimeDay.min()+1))))
    temp_freeTimeNum = pd.DataFrame(np.zeros((24, int(freeTimeDay.max()-freeTimeDay.min()+1))))
    temp_freeTimeMean = pd.DataFrame(np.zeros((24, int(freeTimeDay.max()-freeTimeDay.min()+1))))
    temp_freeTimeStd = pd.DataFrame(np.zeros((24, int(freeTimeDay.max()-freeTimeDay.min()+1))))
    temp_freeTimeString = pd.DataFrame(np.zeros((24, int(freeTimeDay.max()-freeTimeDay.min()+1))))

    # freetime dataframe
    freeTimeNext = 0
    for j in range(int(freeTimeDay.min()), int(freeTimeDay.max())+1):
        for i in range(1,25):
            temp_freeTimeString.iloc[i-1, int(j-freeTimeDay.min()[0])] = i-1
            ind = freeTimeDay[(freeTimeDay[0] == j)&(freeTimeHour[0] == i-1)].index
            temp_freeTimeNum.iloc[i-1, int(j-freeTimeDay.min()[0])] = len(ind)

            if len(ind) == 0:
                freeTimeNext = 0
                continue
            if freeTimeNext != 0:
                ft = pd.concat([pd.DataFrame([freeTimeNext]), freeTimeDuration.loc[ind]])
                freeTimeNext = 0
            else:
                ft = freeTimeDuration.loc[ind]

            if occupiedTime.iloc[ind[-1]+1, 0].hour != i-1:
                year = occupiedTime.iloc[ind[-1], 1].year
                month = occupiedTime.iloc[ind[-1], 1].month
                day = occupiedTime.iloc[ind[-1], 1].day
                if i == 24:
                    day = day + 1
                    i = 0
                    freeTimeNext = (ft.iloc[-1,0] - (datetime(year, month, day, i,0) - occupiedTime.iloc[ind[-1],1]).seconds)
                    ft.iloc[-1,0] = ft.iloc[-1,0] - freeTimeNext
                else:
                    freeTimeNext = (ft.iloc[-1,0] - (datetime(year, month, day, i,0) - occupiedTime.iloc[ind[-1],1]).seconds)
                    ft.iloc[-1,0] = ft.iloc[-1,0] - freeTimeNext

            temp_freeTimeTot.iloc[i-1, int(j-freeTimeDay.min()[0])] = sum(ft[0]/60)
            temp_freeTimeMean.iloc[i-1, int(j-freeTimeDay.min()[0])] = np.mean(ft[0]/60)
            temp_freeTimeStd.iloc[i-1, int(j-freeTimeDay.min()[0])] = np.std(ft[0]/60)
            temp_freetime_df = pd.concat([temp_freeTimeString, temp_freeTimeTot, temp_freeTimeNum, temp_freeTimeMean, temp_freeTimeStd], axis = 1)
            temp_freetime_df.columns = list(range(35))

    # concat dataframe
    temp_freetime = pd.DataFrame()
    for j in range(0,35,7):
        temp = pd.concat([temp_freetime_df[i] for i in temp_freetime_df.columns[j:j+7]]).reset_index(drop = True)
        temp_freetime = pd.concat([temp_freetime, pd.DataFrame(temp)], axis = 1)
    temp_freetime.columns = ['Time', 'Total', 'Num', 'Mean', 'Std']
    
    return temp_freetime

In [ ]:
# 전체 데이터로 freetime만들기 -> 원하는 weight class에 맞는 데이터만 뽑아서 freetime 만들기
# south, north 구분 없음 -> dep_df_south 등으로 바꾸면 할 수 있음 

day = int(input('Select date [22-28] : '))

fig, axs = plt.subplots(5,5, figsize=(50, 50), squeeze = False)

for i, dep_category in enumerate(AC_cat):
    temp_dep_df = dep_df[dep_df['AC'] == f'{dep_category}']
    for j, arr_category in enumerate(AC_cat):
        temp_arr_df = arr_df[arr_df['AC'] == f'{arr_category}']
        
        temp_df = _freetime_(temp_arr_df, temp_dep_df)
        temp_df = temp_df[(day - 22) * 24 : ((day - 21)) * 24]
        axs[j, 4-i].bar(temp_df['Time'], temp_df['Mean'], color='navy', alpha = 0.9)
        axs[j, 4-i].errorbar(temp_df['Time'], temp_df['Mean'], yerr = temp_df['Std'], fmt='1r', capsize=3)
        axs[j, 4-i].axhline(np.mean(temp_df['Mean'].iloc[4:]), xmin = 1/6, color='g', linestyle = '--', linewidth = 2)    # 04-24시 평균 수평선
        axs[j, 4-i].annotate(np.round(np.mean(temp_df['Mean'].iloc[4:]),1), xy = (24, np.mean(temp_df['Mean'].iloc[4:])), xytext = (25, np.mean(temp_df['Mean'].iloc[4:])))
        # sns.barplot(ax = axs[j, 4-i], x = "Time", y = "Mean", data = temp_df)
        axs[j, 4-i].set_title(f'Dep : {dep_category} - Arr : {arr_category}')
fig.suptitle(f'Dec{day}th Runway Freetime paired with Weight Class', fontsize = 80)
plt.savefig(save_dir + f'Dec{day}_rwyFree.png', bbox_inches='tight', pad_inches=1)

In [ ]:
plt.figsize = (30,30)
total_df = _freetime_(arr_df, dep_df)
total_df = total_df[(day - 22) * 24 : ((day - 21)) * 24]
plt.bar(total_df['Time'], total_df['Mean'], color='navy', alpha = 0.9)
plt.errorbar(total_df['Time'], total_df['Mean'], yerr = total_df['Std'], fmt='1r', capsize=3)
plt.axhline(np.mean(total_df['Mean'].iloc[4:]), xmin = 1/6, color='g', linestyle = '--', linewidth = 2)    # 04-24시 평균 수평선
plt.annotate(np.round(np.mean(total_df['Mean'].iloc[4:]),1), xy = (24, np.mean(total_df['Mean'].iloc[4:])), xytext = (25, np.mean(total_df['Mean'].iloc[4:])))
plt.savefig(save_dir + f'Dec{day}_rwyFreeTotal.png', bbox_inches='tight', pad_inches=1)

***
***
***
## METAR

In [ ]:
metar_df = pd.read_csv(f'../input/metar_20191222-20191228.csv', index_col=False, header = 0)
metar_df['IssueTime'] = pd.to_datetime(metar_df['IssueTime'])

In [ ]:
dep_df['DensityAltitude'] = 0
dep_df['Headwind'] = 0
dep_df['Wet'] = 0

In [ ]:
for i in range(len(metar_df)-1):
    dep_df.loc[dep_df['Time'] >= metar_df['IssueTime'][i], 'DensityAltitude'] = metar_df['DensityAltitude'][i]
    dep_df.loc[dep_df['Time'] >= metar_df['IssueTime'][i], 'Headwind'] = metar_df['Headwind'][i]
    dep_df.loc[dep_df['Time'] >= metar_df['IssueTime'][i], 'Wet'] = metar_df['Wet'][i]

In [ ]:
dep_df_north = dep_df[dep_df['RWY'] <= 3]
dep_df_south = dep_df[dep_df['RWY'] > 3]
dep_df_north.loc[:,'AC'] = dep_df_north['AC'].map(mapping)
dep_df_south.loc[:,'AC'] = dep_df_south['AC'].map(mapping)

In [ ]:
dep_df_north.describe()

In [ ]:
dep_df_south.describe()

### Density Altitude

DA는 Northflow, South flow 모두 max -900 정도로 낮은 DA를 보임 -> 50% 지점에서 나누어도 큰 의미 없음

In [ ]:
dep_df_north_highDA = dep_df_north[dep_df_north['DensityAltitude'] >= -1866.010659]
dep_df_north_lowDA = dep_df_north[dep_df_north['DensityAltitude'] < -1866.010659]
dep_df_south_highDA = dep_df_south[dep_df_south['DensityAltitude'] >= -1866.010659]
dep_df_south_lowDA = dep_df_south[dep_df_south['DensityAltitude'] < -1866.010659]

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(2,2,1)
plt.axvline(np.mean(dep_df_north_highDA['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_highDA, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_highDA['DIST'])), 
             xy = (np.mean(dep_df_north_highDA['DIST']), 10),
             xytext = (np.mean(dep_df_north_highDA['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_highDA')

plt.subplot(2,2,2)
plt.axvline(np.mean(dep_df_north_lowDA['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_lowDA, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_lowDA['DIST'])), 
             xy = (np.mean(dep_df_north_lowDA['DIST']), 10),
             xytext = (np.mean(dep_df_north_lowDA['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_lowDA')

plt.subplot(2,2,3)
plt.axvline(np.mean(dep_df_south_highDA['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_highDA, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_highDA['DIST'])), 
             xy = (np.mean(dep_df_south_highDA['DIST']), 10),
             xytext = (np.mean(dep_df_south_highDA['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_highDA')

plt.subplot(2,2,4)
plt.axvline(np.mean(dep_df_south_lowDA['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_lowDA, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_lowDA['DIST'])), 
             xy = (np.mean(dep_df_south_lowDA['DIST']), 10),
             xytext = (np.mean(dep_df_south_lowDA['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_lowDA')

plt.savefig(save_dir + 'densityaltitude_dist.png', bbox_inches='tight', pad_inches=1)

### Headwind

In [ ]:
# 10kts기준

dep_df_north_highHW = dep_df_north[dep_df_north['Headwind'] >= 10]
dep_df_north_lowHW = dep_df_north[dep_df_north['Headwind'] < 10]
dep_df_south_highHW = dep_df_south[dep_df_south['Headwind'] >= 10]
dep_df_south_lowHW = dep_df_south[dep_df_south['Headwind'] < 10]

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(2,2,1)
plt.axvline(np.mean(dep_df_north_highHW['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_highHW, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_highHW['DIST'])), 
             xy = (np.mean(dep_df_north_highHW['DIST']), 10),
             xytext = (np.mean(dep_df_north_highHW['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_highHW')

plt.subplot(2,2,2)
plt.axvline(np.mean(dep_df_north_lowHW['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_lowHW, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_lowHW['DIST'])), 
             xy = (np.mean(dep_df_north_lowHW['DIST']), 10),
             xytext = (np.mean(dep_df_north_lowHW['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_lowHW')

plt.subplot(2,2,3)
plt.axvline(np.mean(dep_df_south_highHW['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_highHW, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_highHW['DIST'])), 
             xy = (np.mean(dep_df_south_highHW['DIST']), 10),
             xytext = (np.mean(dep_df_south_highHW['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_highHW')

plt.subplot(2,2,4)
plt.axvline(np.mean(dep_df_south_lowHW['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_lowHW, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_lowHW['DIST'])), 
             xy = (np.mean(dep_df_south_lowHW['DIST']), 10),
             xytext = (np.mean(dep_df_south_lowHW['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_lowHW')

plt.savefig(save_dir + 'headwind_dist.png', bbox_inches='tight', pad_inches=1)

### Runway Condition

In [ ]:
dep_df_north_Wet= dep_df_north[dep_df_north['Wet'] == 1]
dep_df_north_notWet = dep_df_north[dep_df_north['Wet'] == 0]
dep_df_south_Wet = dep_df_south[dep_df_south['Wet'] == 1]
dep_df_south_notWet = dep_df_south[dep_df_south['Wet'] == 0]

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(2,2,1)
plt.axvline(np.mean(dep_df_north_Wet['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_Wet, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_Wet['DIST'])), 
             xy = (np.mean(dep_df_north_Wet['DIST']), 10),
             xytext = (np.mean(dep_df_north_Wet['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_Wet')

plt.subplot(2,2,2)
plt.axvline(np.mean(dep_df_north_notWet['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_north_notWet, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_north_notWet['DIST'])), 
             xy = (np.mean(dep_df_north_notWet['DIST']), 10),
             xytext = (np.mean(dep_df_north_notWet['DIST']), 0))
plt.legend(labels = labels1)
plt.title('dep_df_north_notWet')

plt.subplot(2,2,3)
plt.axvline(np.mean(dep_df_south_Wet['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_Wet, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_Wet['DIST'])), 
             xy = (np.mean(dep_df_south_Wet['DIST']), 10),
             xytext = (np.mean(dep_df_south_Wet['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_Wet')

plt.subplot(2,2,4)
plt.axvline(np.mean(dep_df_south_notWet['DIST']), color='crimson', linestyle = '--', linewidth = 2)
sns.histplot(data = dep_df_south_notWet, x="DIST", kde=True, hue="RWY", multiple='layer')
plt.annotate(round(np.mean(dep_df_south_notWet['DIST'])), 
             xy = (np.mean(dep_df_south_notWet['DIST']), 10),
             xytext = (np.mean(dep_df_south_notWet['DIST']), 0))
plt.legend(labels = labels2)
plt.title('dep_df_south_Wet')

plt.savefig(save_dir + 'rwycondition_dist.png', bbox_inches='tight', pad_inches=1)

## ML??

In [ ]:
dep_df_north

In [ ]:
# ground speed를 가져오면 도움이 될까?

## Sensitivity Analysis

In [ ]:
import statistics

In [ ]:
for i in list(set(dep_df_north['AC'].values))[1:]:
    print(f'{str(i)} : ' , statistics.stdev(dep_df_north[dep_df_north['AC'] == i]['DIST']))    # 무게별로 sd


In [ ]:
sns.scatterplot(data=dep_df_north, x="Headwind", y="DIST", hue = 'AC')